In [88]:
import pandas as pd
import json
import os
import numpy as np
import time
from datetime import timedelta

import importlib

from pymongo import MongoClient

client = MongoClient('localhost', 27017) # change the ip and port to your mongo database's
db = client['kagglecrawler']
search_log_coll = db['search_log']
download_log_coll = db['download_log']
raw_notebook_coll = db['raw_notebooks']
task_log_coll = db['task_log']
central_task_log_coll = db['central_task_log']
notebook_metadata_coll = db['notebook_metadata']
old_notebook_metadata_coll = db['notebook_metadata_coll']
QUERY_FILE = os.path.join(os.getcwd(), 'Queries/pwc_queries.csv')


In [96]:
df_search_log = pd.DataFrame.from_dict(list(search_log_coll.find()))
df_task_log = pd.DataFrame.from_dict(list(task_log_coll.find()))
df_download_log = pd.DataFrame.from_dict(list(download_log_coll.find()))


In [55]:
queries = list(df_task_log['query'])
query_types = []
for query in queries: 
    query_types.append(type(query))


In [98]:
df_download_log

,_id,query,title,kernel_ref
0,6389270d6bba1b197c8214da,MinCut Pooling,Mincut Pooling in Graph Neural Networks,elmahy/mincut-pooling-in-graph-neural-networks
1,6389270d6bba1b197c8214db,Residual GRU,Transformers: report on Attention Is All You Need,samuelnordmann/transformers-report-on-attentio...
2,6389270d6bba1b197c8214dc,Residual GRU,Complete+EDA+Glove+RNN+LSTM+GRU+Birectional+BERT,krishnamore/complete-eda-glove-rnn-lstm-gru-bi...
3,6389270d6bba1b197c8214dd,Residual GRU,ML & DL Tutorials,gauravmehta13/ml-dl-tutorials
4,6389270d6bba1b197c8214de,Residual GRU,NER_BiLSTM_model,trangdothuy/ner-bilstm-model
...,...,...,...,...
1420,6389270d6bba1b197c821a66,RAE,Relative Age Effect in the American Baseball,gerosa/relative-age-effect-in-the-american-bas...
1421,6389270d6bba1b197c821a67,GBO,Elo Data Cleaning and EDA,brendanhasz/elo-data-cleaning-and-eda
1422,6389270d6bba1b197c821a68,Graph Representation with Global structure,"Starter: 18,393 Pitchfork Reviews afc5a92b-2",maxkleiner/starter-18-393-pitchfork-reviews-af...
1423,6389270d6bba1b197c821a69,CoaT,shoes_prep the data,hanyan/shoes-prep-the-data


In [77]:
def drop_error(x): 
    if type(x)==list: 
        return True
    else: 
        return False
df_task_log['drop_error'] = df_task_log['query'].apply(drop_error)

In [82]:
clean_task = df_task_log.loc[df_task_log['drop_error']==False]

In [94]:
clean_task

,query,searched,downloaded
0,Gravity,105,9
1,Discrete Cosine Transform,35,7
2,Gated Linear Network,5,3
3,Weight Decay,11,1
4,BiGG,145,3
...,...,...,...
1255,UBOFAB19,0,-1
1256,PASCAL Face,48,-1
1257,ASPEC,76,-1
1258,Sub-Slot Dialogue dataset id number domain,1,-1


In [6]:
from kagglecrawler.distributed_notebook_crawling import KaggleNotebookCrawler

# Create crawler
kwargs = {
    'search_log_coll': search_log_coll, 
    'download_log_coll': download_log_coll, 
    'central_log_coll': download_log_coll, 
    'task_log_coll': task_log_coll, 
    'raw_notebook_coll': raw_notebook_coll, 
    'notebook_metadata_coll': notebook_metadata_coll
}
crawler = KaggleNotebookCrawler(**kwargs)